### Librerias

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

c:\Users\inesc\Bootcamp\Module_3\Smartwaste_Streamlit\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Fuentes RAG

In [10]:
url = 'https://www.madrid.es/UnidadesDescentralizadas/Educacion_Ambiental/ContenidosBasicos/Publicaciones/Residuos_Limpieza/GuiaReducirResiduos/GuíaBuenasPrácticasReducirResiduos.pdf'
url2 = 'DATA/CS-RecogidaResiduos_Folleto2024.pdf'
url3 = 'https://www.madrid.es/UnidadesDescentralizadas/UDCEstadistica/Nuevaweb/Publicaciones/anuesta/nuevos/Anuario%20Estadístico%20Municipal/Anuario%20estadístico%202023/Capítulos/CAPITULO%206.pdf'
url4 = 'https://www.icv.csic.es/prevencion/Documentos/residuos/puntos.pdf'
url5 =  'https://www.redcimas.org/wordpress/wp-content/uploads/2013/02/gestionresiduos_comparativa.pdf'
url6 = 'https://conerrede.es/wp-content/uploads/2022/08/dossier-informativo_ayto-madrid.pdf'


### Configuracion

In [12]:
API_KEY = ''
llm = ChatOpenAI(model='gpt-3.5-turbo-0125' , temperature= 0 , api_key=API_KEY)
emmbeding = OpenAIEmbeddings(api_key= API_KEY)

### Corpus

In [13]:
urls = [url, url3, url4,url5,url6] #un libro de libros

In [16]:

pages = []
for url in urls:
    pdf = PyPDFLoader(url)
    pages += pdf.load_and_split()

In [17]:
len(pages)

222

In [18]:
pages[210]

Document(metadata={'source': 'https://conerrede.es/wp-content/uploads/2022/08/dossier-informativo_ayto-madrid.pdf', 'page': 13, 'page_label': '14'}, page_content='La prevención de residuos \nNo sólo es necesario separar correctamente los residuos generados sino incorporar nuevos hábitos \nde consumo más sostenibles y responsables encaminados a la reducción de la producción de \nresiduos como:  \n• Evitar el consumo de productos de un solo uso (bolsas de plástico, botellas de plástico, film \ntransparente, papel de aluminio, vasos y platos desechables , servilletas de papel etc.) y \nsustituirlos por envases reutilizables y/o rellenables (bolsas de tela o rafia, servilletas de \ntela, cantimploras, portabocadillos, recipientes reutilizables etc.  \n• Elegir productos a granel, o con el mínimo de embalaje posibl e y procurar evitar aquellos \ncon embalajes compuestos de distintos materiales, ya que resulta un incremento de \nresiduos y resulta difícil separarlos.  \n• Reutilizar los prod

# Vectorstore 

In [33]:
from langchain.vectorstores import Chroma

# Initialize the vectorstore with persistence
vectorstore = Chroma.from_documents(
    documents=pages,
    embedding=emmbeding,
    persist_directory="C:/Users/inesc/Bootcamp/Module_3/Smartwaste_Streamlit/vector_store"  # Specify directory
)

# Persist the vectorstore to disk
vectorstore.persist()

# Create a retriever
retriever = vectorstore.as_retriever()


# Prompt Template

In [34]:
 template = """
          ###INSTRUCCIONES:
          Eres un asistente virtual en tareas de concienciacion del reciclaje y las buenas practicas medioambientales y tu objetivo es recopilar y dar respuesta segun la informacion que tienes a las preguntas que te planteen los usuarios.Seimpre con un tono amigable y pedagogico.

          En tu respuesta, SIEMPRE:
          (0)Se un lector atento a los detalles: Lee la pregunta y el contexto, entiende ambos antes de responder
          (1)Tu respuesta comienza con un tono amigable, asegúrate de que entiendes la pregunta del usuario correctamente, en caso negativo, pregunta al usuario para obtener más información.
          (2)Si el contexto te permite responder a la pregunta, escribe una respuesta detallada, útil y fácil de entender, CON FUENTES REFERENCIADAS EN EL TEXTO. SI NO: no puedes encontrar la respuesta, responde con una explicación, comenzando con: "No pude encontrar la información en los documentos a los que tengo acceso".
          (3)Debajo de tu respuesta, proporciona un enlace a la fuente referenciada( Es decir, párrafos legales, artículos, etc.) y una breve explicación de por qué la fuente es relevante.
          (4)Ahora que ya tienes tu respuesta, revísala para asegurarte de que sea clara, precisa y educada y cumple con los puntos anteriores.

          ###PIENSA PASO POR PASO

            Responde a la siguiente pregunta utilizando el contexto proporcionado.
              ### Question: {question} ###
              ### Context: {context} ###
              ### Respuesta 'util con Fuentes:        
"""

prompt = PromptTemplate.from_template(template)

## Chain

In [35]:
chain = (
    {'context':retriever,
     'question':RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)
 

In [36]:
def get_asnwer(question):
    return chain.invoke(question)

In [37]:
get_asnwer('donde puedo tirar resoduos organicos')

'¡Hola! Para desechar residuos orgánicos, como restos de alimentos, posos de café, hojas y ramas de plantas, entre otros, debes depositarlos en el contenedor gris con tapa naranja. Este contenedor está destinado para los restos orgánicos, mientras que otros residuos domésticos deben ser depositados en lugares como el Punto Limpio Móvil, Punto Limpio Fijo, contenedores de pilas, entre otros.\n\nLa materia orgánica depositada en este contenedor puede ser transformada en compost, un abono orgánico de alta calidad que puede ser utilizado para fertilizar el suelo. En los Centros de tratamiento del Parque Tecnológico de Valdemingómez, se lleva a cabo el compostaje de la materia orgánica para obtener este valioso abono.\n\nSi tienes dudas sobre dónde desechar un residuo en particular, se recomienda depositarlo en el contenedor gris con tapa naranja.\n\nFuente: https://www.madrid.es/UnidadesDescentralizadas/Educacion_Ambiental/ContenidosBasicos/Publicaciones/Residuos_Limpieza/GuiaReducirResidu

In [27]:
iface = gr.Interface(fn=get_asnwer, inputs=gr.Textbox(
    value="Pregúntame lo que necesites"),
    live=False, 
    outputs="markdown",  
    title="Inteligencia sobre documentación mediante LLM",
    description="Puedes preguntar cualquier pregunta sobre migración",
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()

c:\Users\inesc\Bootcamp\Module_3\Smartwaste_Streamlit\venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
